# Model Calibration with Epydemix (R Version)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ngozzi/tech-transfer-epdemix/blob/main/sessions/session-3/r-colab/02_calibration_tutorial.ipynb)

This tutorial covers calibrating epidemic models to observed data using Approximate Bayesian Computation (ABC) methods—all using R via the `reticulate` package.

**What you'll learn:**
- Set up calibration with prior distributions and distance functions
- Compare three ABC calibration strategies: ABC-SMC, ABC Rejection, and Top X%
- Visualize posterior distributions and calibration fits
- Run projections from calibrated models

In [ ]:
# Colab installation (skip if running locally)
!pip install epydemix
%load_ext rpy2.ipython

In [ ]:
%%R
# Install and load reticulate
if (!require("reticulate", quietly = TRUE)) {
  install.packages("reticulate")
}
library(reticulate)
use_python("/usr/bin/python3", required = TRUE)
py_config()

---
## 1. Loading and Exploring the Data

We'll calibrate an SIR model to synthetic incidence data (new infections per day) from Indonesia. The data was generated from an SIR model with known parameters, plus observation noise.

In [ ]:
%%R
library(readr)
library(ggplot2)
library(dplyr)

# Load the CSV directly from GitHub
data <- read_csv(
  "https://raw.githubusercontent.com/epistorm/epydemix/refs/heads/main/tutorials/data/incidence_data.csv",
  show_col_types = FALSE
)

# Ensure "date" is a Date object
data <- data %>% mutate(date = as.Date(date))

# Split into calibration and projection periods
n_total <- nrow(data)
n_calibration <- n_total - 40
data_calibration <- data[1:n_calibration, ]
data_projection <- data[(n_calibration + 1):n_total, ]

cat(sprintf("Calibration period: %s to %s (%d days)\n",
            data_calibration$date[1],
            data_calibration$date[nrow(data_calibration)],
            nrow(data_calibration)))
cat(sprintf("Projection period: %s to %s (%d days)\n",
            data_projection$date[1],
            data_projection$date[nrow(data_projection)],
            nrow(data_projection)))

In [ ]:
%%R
# Plot the data
ggplot() +
  geom_point(data = data_calibration, aes(x = date, y = data), color = "black") +
  geom_point(data = data_projection, aes(x = date, y = data), color = "gray", alpha = 0.5) +
  geom_vline(xintercept = as.numeric(data_calibration$date[nrow(data_calibration)]),
             color = "red", linetype = "dashed", alpha = 0.5) +
  labs(x = "Date", y = "New Infections", title = "Incidence Data: Calibration vs Held-out") +
  theme_minimal()

---
## 2. Setting Up the Model

We'll use a predefined SIR model with Indonesia's population data.

In [ ]:
%%R
# Import epydemix
epydemix <- import("epydemix")
load_predefined_model <- epydemix$load_predefined_model

# Load predefined SIR model
model <- load_predefined_model("SIR")
model$import_epydemix_population(population_name = "Indonesia")

model

In [ ]:
%%R
# Import utils
utils_module <- import("epydemix.utils")
compute_simulation_dates <- utils_module$compute_simulation_dates

# Get population size
Nk_r <- py_to_r(model$population$Nk)

# Initial conditions: 0.05% of population infected
infected <- as.integer(Nk_r * 0.05 / 100)
susceptible <- as.integer(Nk_r - infected)
recovered <- rep(0L, length(Nk_r))

initial_conditions <- reticulate::dict(
  Susceptible = susceptible,
  Infected = infected,
  Recovered = recovered
)

# Simulation dates
start_date_chr <- as.character(data_calibration$date[1])
end_date_calibration_chr <- as.character(data_calibration$date[nrow(data_calibration)])
end_date_projection_chr <- as.character(data_projection$date[nrow(data_projection)])

# Parameters dict for calibration
parameters <- reticulate::dict(
  initial_conditions_dict = initial_conditions,
  epimodel = model,
  start_date = start_date_chr,
  end_date = end_date_calibration_chr
)

# Compute simulation dates
simulation_dates_calibration <- compute_simulation_dates(
  start_date = start_date_chr,
  end_date = end_date_calibration_chr
)

simulation_dates_projection <- compute_simulation_dates(
  start_date = start_date_chr,
  end_date = end_date_projection_chr
)

---
## 3. Defining Priors and the Calibration Wrapper

### Prior Distributions

Prior distributions encode our beliefs about parameter values before seeing the data. We use `scipy.stats` distributions:

In [ ]:
%%R
# Import scipy.stats
scipy_stats <- import("scipy.stats")

# Uniform priors for transmission and recovery rates
priors <- reticulate::dict(
  transmission_rate = scipy_stats$uniform(loc = 0.010, scale = 0.020),  # U(0.01, 0.03)
  recovery_rate = scipy_stats$uniform(loc = 0.15, scale = 0.1)          # U(0.15, 0.25)
)

priors

### Simulation Wrapper Function

The `ABCSampler` requires a function that returns a dictionary with a `'data'` key containing the quantity to compare against observations. This wrapper extracts new infections (S→I transitions) from the simulation output.

**Note:** We define this wrapper in Python since it needs to be callable by the Python calibration engine.

In [ ]:
%%R
# Import calibration module
calib <- import("epydemix.calibration")
ABCSampler <- calib$ABCSampler
rmse <- calib$rmse

# Define the wrapper function in Python (required for ABCSampler)
py_run_string("
from epydemix import simulate

def simulate_wrapper(parameters):
    results = simulate(**parameters)
    return {'data': results.transitions['Susceptible_to_Infected_total']}
")

simulate_wrapper <- py$simulate_wrapper

# Observed data as R numeric vector
observed_data <- data_calibration$data

# Initialize the ABC sampler
abc_sampler <- ABCSampler(
  simulation_function = simulate_wrapper,
  priors = priors,
  parameters = parameters,
  observed_data = observed_data,
  distance_function = rmse  # Root Mean Squared Error
)

abc_sampler

---
## 4. Running Calibration

Epydemix supports three calibration strategies:

| Method | Description | When to Use |
|--------|-------------|-------------|
| **ABC-SMC** | Sequential Monte Carlo with adaptive tolerance | Most accurate, recommended for final results |
| **ABC Rejection** | Simple accept/reject with fixed tolerance | Fast exploratory analysis |
| **Top X%** | Keep best fraction of simulations | Fixed runtime, good for prototyping |

Let's run all three and compare.

In [ ]:
%%R
# ABC-SMC: 5 generations, 100 particles per generation
results_abc_smc <- abc_sampler$calibrate(
  strategy = "smc",
  num_particles = 100L,
  num_generations = 5L
)

In [ ]:
%%R
# ABC Rejection: fixed tolerance based on ABC-SMC final epsilon
results_abc_rejection <- abc_sampler$calibrate(
  strategy = "rejection",
  num_particles = 100L,
  epsilon = 550000
)

In [ ]:
%%R
# Top X%: keep best 10% of 1000 simulations
results_top_perc <- abc_sampler$calibrate(
  strategy = "top_fraction",
  Nsim = 1000L,
  top_fraction = 0.1
)

---
## 5. Comparing Calibration Results

### Calibration Fits

Let's visualize how well each method fits the observed data:

In [ ]:
%%R
# Import visualization functions
viz <- import("epydemix.visualization")
plot_quantiles <- viz$plot_quantiles

# Get quantiles for each method
df_quantiles_smc <- results_abc_smc$get_calibration_quantiles(dates = simulation_dates_calibration)
df_quantiles_rejection <- results_abc_rejection$get_calibration_quantiles(dates = simulation_dates_calibration)
df_quantiles_top <- results_top_perc$get_calibration_quantiles(dates = simulation_dates_calibration)

# Plot ABC-SMC
plot_quantiles(
  df_quantiles_smc,
  columns = "data",
  data = data_calibration,
  title = "ABC-SMC",
  colors = "red",
  show_data = TRUE,
  labels = list("New Infections")
)

In [ ]:
%%R
# Plot ABC Rejection
plot_quantiles(
  df_quantiles_rejection,
  columns = "data",
  data = data_calibration,
  title = "ABC Rejection",
  colors = "blue",
  show_data = TRUE
)

In [ ]:
%%R
# Plot Top 10%
plot_quantiles(
  df_quantiles_top,
  columns = "data",
  data = data_calibration,
  title = "Top 10%",
  colors = "green",
  show_data = TRUE
)

### Posterior Distributions

The posterior distribution shows our updated beliefs about parameter values after seeing the data. Let's compare the 2D joint posteriors:

In [ ]:
%%R
# Import 2D posterior plotting
plot_posterior_distribution_2d <- viz$plot_posterior_distribution_2d

# ABC-SMC posterior
plot_posterior_distribution_2d(
  results_abc_smc$get_posterior_distribution(),
  "transmission_rate",
  "recovery_rate",
  kind = "kde",
  title = "ABC-SMC",
  prior_range = FALSE
)

In [ ]:
%%R
# ABC Rejection posterior
plot_posterior_distribution_2d(
  results_abc_rejection$get_posterior_distribution(),
  "transmission_rate",
  "recovery_rate",
  kind = "kde",
  title = "ABC Rejection",
  prior_range = FALSE
)

In [ ]:
%%R
# Top 10% posterior
plot_posterior_distribution_2d(
  results_top_perc$get_posterior_distribution(),
  "transmission_rate",
  "recovery_rate",
  kind = "kde",
  title = "Top 10%",
  prior_range = FALSE
)

We can also examine the marginal posterior distributions for each parameter:

In [ ]:
%%R
# Import marginal posterior plotting
plot_posterior_distribution <- viz$plot_posterior_distribution

# Transmission rate - all methods on one plot
ax <- plot_posterior_distribution(
  results_abc_smc$get_posterior_distribution(),
  "transmission_rate",
  kind = "kde",
  color = "red",
  label = "ABC-SMC",
  prior_range = FALSE
)

ax <- plot_posterior_distribution(
  results_abc_rejection$get_posterior_distribution(),
  "transmission_rate",
  kind = "kde",
  color = "blue",
  label = "ABC Rejection",
  prior_range = FALSE,
  ax = ax
)

ax <- plot_posterior_distribution(
  results_top_perc$get_posterior_distribution(),
  "transmission_rate",
  kind = "kde",
  color = "green",
  label = "Top 10%",
  prior_range = FALSE,
  ax = ax
)

ax$set_title("Transmission Rate Posterior")
ax$legend()
ax

In [ ]:
%%R
# Recovery rate - all methods on one plot
ax <- plot_posterior_distribution(
  results_abc_smc$get_posterior_distribution(),
  "recovery_rate",
  kind = "kde",
  color = "red",
  label = "ABC-SMC",
  prior_range = FALSE
)

ax <- plot_posterior_distribution(
  results_abc_rejection$get_posterior_distribution(),
  "recovery_rate",
  kind = "kde",
  color = "blue",
  label = "ABC Rejection",
  prior_range = FALSE,
  ax = ax
)

ax <- plot_posterior_distribution(
  results_top_perc$get_posterior_distribution(),
  "recovery_rate",
  kind = "kde",
  color = "green",
  label = "Top 10%",
  prior_range = FALSE,
  ax = ax
)

ax$set_title("Recovery Rate Posterior")
ax$legend()
ax

### Distance Distributions

The distribution of distances (RMSE) for accepted particles shows how well each method concentrates on good-fitting parameters:

In [ ]:
%%R
# Import distance distribution plotting
plot_distance_distribution <- viz$plot_distance_distribution

# Plot all methods on one figure
ax <- plot_distance_distribution(
  results_abc_smc$get_distances(),
  kind = "kde",
  color = "red",
  label = "ABC-SMC",
  xlabel = "RMSE"
)

ax <- plot_distance_distribution(
  results_abc_rejection$get_distances(),
  kind = "kde",
  color = "blue",
  label = "ABC Rejection",
  xlabel = "RMSE",
  ax = ax
)

ax <- plot_distance_distribution(
  results_top_perc$get_distances(),
  kind = "kde",
  color = "green",
  label = "Top 10%",
  xlabel = "RMSE",
  ax = ax
)

ax$set_title("Distribution of Distances for Accepted Particles")
ax$legend()
ax

---
## 6. Running Projections

After calibrating the model, we can project the epidemic forward in time. The `run_projections` method uses the posterior parameter samples to generate future trajectories with uncertainty quantification.

In [ ]:
%%R
# Create projection parameters (extend end date)
projection_parameters <- reticulate::dict(
  initial_conditions_dict = initial_conditions,
  epimodel = model,
  start_date = start_date_chr,
  end_date = end_date_projection_chr
)

# Run projections using the calibrated posterior
results_with_projections <- abc_sampler$run_projections(projection_parameters)

In [ ]:
%%R
# Get quantiles for calibration and projection periods
df_calibration <- results_with_projections$get_calibration_quantiles(simulation_dates_calibration)
df_projection <- results_with_projections$get_projection_quantiles(simulation_dates_projection)

# Plot calibration fit
ax <- plot_quantiles(
  df_calibration,
  columns = "data",
  colors = "orange",
  show_data = FALSE,
  labels = list("Calibration fit")
)

# Plot projection
ax <- plot_quantiles(
  df_projection,
  columns = "data",
  colors = "blue",
  show_data = FALSE,
  labels = list("Projection"),
  ax = ax
)

# Add observed data points using matplotlib
plt <- import("matplotlib.pyplot")

ax$plot(
  as.character(data_calibration$date),
  data_calibration$data,
  "ko",
  markersize = 4L,
  label = "Calibration data"
)

ax$plot(
  as.character(data_projection$date),
  data_projection$data,
  "o",
  color = "gray",
  markersize = 4L,
  alpha = 0.6,
  label = "Held-out data"
)

# Mark calibration cutoff
ax$axvline(
  as.character(data_calibration$date[nrow(data_calibration)]),
  color = "red",
  linestyle = "--",
  alpha = 0.5
)

ax$set_ylabel("New Infections")
ax$set_title("Model Calibration and Projection")
ax$legend(loc = "upper right")
ax

---
## Resources

- [Epydemix Documentation](https://epydemix.readthedocs.io/)
- [ABC Methods Reference (Minter et al., 2019)](https://www.sciencedirect.com/science/article/pii/S175543651930026X)
- [ABC-SMC Algorithm (Toni et al., 2009)](https://pubmed.ncbi.nlm.nih.gov/19205079)
- [Reticulate Package](https://rstudio.github.io/reticulate/)